# Organizational webhook receiver


When the triggering event occurs, the webhook will inject the JSON payload into this notebook as a variable called `webhookPayload` and run the notebook. Do not declare a value for `webhookPayload`, as that definition will overwrite the value injected by the webhook.

## Log webhook payload

This cell is for testing webhooks, can be safely deleted for production webhooks

In [35]:
import json
from datetime import datetime

dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
with open(f"/arcgis/home/webhookoutput.json", "a") as f:
    payload_string = json.dumps(webhookPayload, indent=2, sort_keys=True)
    f.write(dt + "\n")
    f.write(payload_string + "\n")
    f.write("======================\n")
    

## Connect to portal

In [ ]:
from arcgis.gis import GIS
gis = GIS("home")

## Parse payload

See [this documentation](https://enterprise.arcgis.com/en/portal/latest/administer/windows/webhook-payloads.htm) for more information

In [18]:
for event in webhookPayload["events"]:
    
    # User that triggered the event
    username = event["username"]
    user = gis.users.get(username)
    
    # Time the event occurred (Unix epoch milliseconds)
    timestamp = event["when"]
    
    # Operation type performed by the triggering user
    operation = event["operation"]
    
    # Resource type on which the operation was performed. Can be item, group, or user
    resource_type = event["source"]
    
    # ID of the resource affected by the operation
    resource_id = event["id"]
    
    # Affected resource
    if resource_type == "item":
        resource = gis.content.get(resource_id)
    elif resource_type == "group":
        resource = gis.groups.get(resource_id)
    elif resource_type == "user":
        resource = gis.users.get(resource_id)
    else: 
        resource = None
    
    # Additional properties dictionary associated with the event trigger
    properties = event["properties"]